In [1]:
from collections import defaultdict
from random import uniform
from math import sqrt
import os
import csv
import numpy as np
#import seaborn as sns
import pandas as pd
import mindspore as ms
import matplotlib.pyplot as plt
from mindspore import Tensor,nn, context
from mindspore.ops import operations as ops
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")


def point_avg(points):
    """
    Accepts a list of points, each with the same number of dimensions.
    NB. points can have more dimensions than 2
    
    Returns a new point which is the center of all the points.
    """
    dimensions = len(points[0])

    new_center = []

    for dimension in range(dimensions):
        dim_sum = 0  # dimension sum
        for p in points:
            dim_sum += p[dimension]

        # average of each dimension
        new_center.append(dim_sum / float(len(points)))

    return new_center


def update_centers(data_set, assignments):
    """
    Accepts a dataset and a list of assignments; the indexes 
    of both lists correspond to each other.
    Compute the center for each of the assigned groups.
    Return `k` centers where `k` is the number of unique assignments.
    """
    new_means = defaultdict(list)
    centers = []
    for assignment, point in zip(assignments, data_set):
        new_means[assignment].append(point)
        
    for points in new_means.values():
        centers.append(point_avg(points))

    return centers


def assign_points(data_points, centers):
    """
    Given a data set and a list of points betweeen other points,
    assign each point to an index that corresponds to the index
    of the center point on it's proximity to that point. 
    Return a an array of indexes of centers that correspond to
    an index in the data set; that is, if there are N points
    in `data_set` the list we return will have N elements. Also
    If there are Y points in `centers` there will be Y unique
    possible values within the returned list.
    """
    assignments = []
    for point in data_points:
        shortest = 100000  # positive infinity
        shortest_index = 0
        for i in range(len(centers)):
            val = distance(point, centers[i])
            if val < shortest:
                shortest = val
                shortest_index = i
        assignments.append(shortest_index)
    return assignments


def distance(a, b):
    """
    """
    dimensions = len(a)
    
    _sum = 0
    for dimension in range(dimensions):
        difference_sq = (a[dimension] - b[dimension]) ** 2
        _sum += difference_sq
    return sqrt(_sum)


def generate_k(data_set, k):
    """
    Given `data_set`, which is an array of arrays,
    find the minimum and maximum for each coordinate, a range.
    Generate `k` random points between the ranges.
    Return an array of the random points within the ranges.
    """
    centers = []
    dimensions = len(data_set[0])
    min_max = defaultdict(int)

    for point in data_set:
        for i in range(dimensions):
            val = point[i]
            min_key = 'min_%d' % i
            max_key = 'max_%d' % i
            if min_key not in min_max or val < min_max[min_key]:
                min_max[min_key] = val
            if max_key not in min_max or val > min_max[max_key]:
                min_max[max_key] = val

    for _k in range(k):
        rand_point = []
        for i in range(dimensions):
            min_val = min_max['min_%d' % i]
            max_val = min_max['max_%d' % i]
            
            rand_point.append(uniform(min_val, max_val))

        centers.append(rand_point)

    return centers


def k_means(dataset, k):
    k_points = generate_k(dataset, k)
    assignments = assign_points(dataset, k_points)
    old_assignments = None
    noo=0
    while noo<10:
        new_centers = update_centers(dataset, assignments)
        old_assignments = assignments
        assignments = assign_points(dataset, new_centers)
        noo+=1
        print(noo)
    return assignments


In [2]:
#data load
import scipy.io as scio
letter_data = scio.loadmat('./data/le1.mat')
letter_msg=letter_data['data']
letter_label=letter_data['label']
letter_label[0][0]

20

In [3]:
qz=k_means(letter_msg,26)

1
2
3
4
5
6
7
8
9
10


In [25]:
index=[]
for i in range(26):
    p=np.zeros(26)
    index.append(p)
for i in range(len(letter_msg)):
    index[letter_label[i][0]-1][qz[i]]+=1
total=0
for i in range(26):
    total+=max(index[i])
total

6804.0

In [20]:
k_res=qz
for i in range(20000):
    k_res[i]+=1

In [21]:
#外部指标计算
a=0
b=0
c=0
d=0
for i in range(20000):
    for j in range(20000):
        if letter_label[i]==letter_label[j]:
            if k_res[i]==k_res[j]:
                a+=1
            else:
                b+=1
        else:
            if k_res[i]==k_res[j]:
                c+=1
            else:
                d+=1
print('{}.{}.{}.{}'.format(a,b,c,d))

3309466.12088576.27938950.356663008


In [22]:
JC=a/(a+b+c)
JC

0.07636584468068296

In [23]:
FMI=np.sqrt((a/(a+b))*(a/(a+c)))
FMI

0.15087287629013182

In [24]:
RI=((a+d))/(a+b+c+d)
RI

0.899931185

In [27]:
purity=total/20000
purity

0.3402